In [1]:
import os

# os.environ['GITHUB_ACCESS_TOKEN'] = ''

# print(os.getenv("GITHUB_ACCESS_TOKEN", None))

from lean_dojo import *

/home/mcwave/anaconda3/envs/lean3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-06 11:41:28,761	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Repo containing both mathlib4 and Math-in-lean, fetched in March 2024
# Pickled at '/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl'
# repo = LeanGitRepo(
#     "https://github.com/xiaoxin-yin/math-in-lean",
#     "20077bcd4392317ddb9605404fda3a85e40e8956"
# )

# Repo Mathlib4 on 6/17/24
repo = LeanGitRepo(
    "https://github.com/leanprover-community/mathlib3",
    "65a1391a0106c9204fe45bc73a039f056558cb83",
)

repo

LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83')

In [ ]:
import pickle

fin = open('/home/mcwave/code/axiomatization/datasets/traced_repo_mathlib3_20240831.pkl', 'rb')
traced_repo = pickle.load(fin)
fin.close()

/home/mcwave/anaconda3/envs/lean3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-06 11:41:19,052	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
# import pickle

# fin = open('/home/mcwave/code/automath/atp/datasets/train_traced_theorems_repo_math_in_lean.pkl', 'rb')
# train_traced_theorems = pickle.load(fin)
# fin.close()

In [ ]:
# Save traced_theorems to a pickle file since they are only data we need

import json

def generate_train_file_theorems(train_tac_templates_path, output_path):
    fin = open(train_tac_templates_path, 'r')
    train_tac_templates = json.load(fin)
    fin.close()
    #
    train_file_theorems = {}
    for i in range(len(train_tac_templates)):
        full_name = train_tac_templates[i][2]
        file_path = train_tac_templates[i][3]
        if file_path not in train_file_theorems:
            train_file_theorems[file_path] = set()
        train_file_theorems[file_path].add(full_name)
    #
    train_traced_theorems = dict()
    for file_path, full_names in train_file_theorems.items():
        traced_file = traced_repo.get_traced_file(file_path)
        premises = traced_file.get_premise_definitions()
        results = []
        for premise in premises:
            full_name = premise['full_name']
            if full_name in full_names:
                thm = traced_file.get_traced_theorem(full_name)
                thm.comments.insert(0, premise['code'])
                if file_path not in train_traced_theorems:
                    train_traced_theorems[file_path] = dict()
                train_traced_theorems[file_path][full_name] = thm
    #
    for file_path, full_names in train_file_theorems.items():
        if file_path not in train_traced_theorems:
            print(file_path, "not found!")
            continue
        for full_name in full_names:
            if full_name not in train_traced_theorems[file_path]:
                print(full_name, "not found in", file_path)
    #
    fout = open(output_path, 'wb')
    pickle.dump(train_traced_theorems, fout)
    fout.close()


train_tac_templates_path = '/home/mcwave/code/axiomatization/lean3/datasets/tac_templates_in_files/mathlib_tac_templates.json'
output_path = '/home/mcwave/code/axiomatization/lean3/datasets/train_traced_theorems_repo_mathlib3.pkl'
generate_train_file_theorems(train_tac_templates_path, output_path)


In [3]:
# # Exclude processed theoresm from train_traced_theorems
# import os
# import pickle
# from lean_dojo import *

# def get_all_theorems_processed(folder_paths, verbose=True):
#     # Enumerate all .pkl files in the folder
#     output = {}
#     for folder_path in folder_paths:
#         for filename in os.listdir(folder_path):
#             if filename.endswith(".pkl"):
#                 found = False
#                 file_path = os.path.join(folder_path, filename)
#                 fin = open(file_path, 'rb')
#                 while True:
#                     try:
#                         result = pickle.load(fin)
#                     except Exception as e:
#                         break
#                     file_path, full_name, theorem, state_pair = result
#                     if file_path not in output:
#                         output[file_path] = set()
#                     output[file_path].add(full_name)
#                     found = True
#                 fin.close()
#                 if verbose and found:
#                     print(filename, len(output[file_path]), "theorems")
#     return output

# fin = open('/home/mcwave/code/axiomatization/lean3/datasets/train_traced_theorems_repo_mathlib3.pkl', 'rb')
# train_traced_theorems = pickle.load(fin)
# fin.close()
    
# previous_output_paths = [
#     '/home/mcwave/code/axiomatization/lean3/datasets/provability/rag_20240614',
# ]

# print("Getting previously computed theorems")
# processed_theorems = get_all_theorems_processed(previous_output_paths)
# remaining_traced_theorems = dict()
# for train_file_path, traced_theorems in train_traced_theorems.items():
#     if train_file_path not in processed_theorems:
#         remaining_traced_theorems[train_file_path] = traced_theorems
#         continue
#     remaining_theorems = dict()
#     for full_name, thm in traced_theorems.items():
#         if full_name not in processed_theorems[train_file_path]:
#             remaining_theorems[full_name] = thm
#         else:
#             remaining_theorems = dict()
#     print(train_file_path, len(traced_theorems), "->", len(remaining_theorems))
#     remaining_traced_theorems[train_file_path] = remaining_theorems

# fout = open('/home/mcwave/code/automath/atp/datasets/remaining_traced_theorems_repo_math_in_lean_20240617.pkl', 'wb')
# pickle.dump(remaining_traced_theorems, fout)
# fout.close()

Getting previously computed theorems
mathlib__Mathlib__GroupTheory__Index.lean.pkl 1 theorems
mathlib__Mathlib__Data__Set__List.lean.pkl 3 theorems
mathlib__Mathlib__Algebra__MonoidAlgebra__Degree.lean.pkl 6 theorems
solutions__Solutions_S02_Metric_Spaces.lean.pkl 1 theorems
mathlib__Mathlib__Algebra__Order__Group__MinMax.lean.pkl 4 theorems
mathlib__Mathlib__Algebra__Module__Submodule__Lattice.lean.pkl 7 theorems
mathlib__Mathlib__Analysis__Calculus__Deriv__Basic.lean.pkl 24 theorems
mathlib__Mathlib__Logic__Equiv__Basic.lean.pkl 27 theorems
mathlib__Mathlib__MeasureTheory__Function__LpSpace.lean.pkl 6 theorems
mathlib__Mathlib__Analysis__SpecialFunctions__Integrals.lean.pkl 1 theorems
mathlib__Mathlib__Data__Set__Accumulate.lean.pkl 2 theorems
mathlib__Mathlib__RingTheory__FractionalIdeal__Operations.lean.pkl 14 theorems
mathlib__Mathlib__GroupTheory__Perm__Cycle__Type.lean.pkl 24 theorems
mathlib__Mathlib__RingTheory__LaurentSeries.lean.pkl 2 theorems
mathlib__Mathlib__Logic__Small_

mathlib__Mathlib__Data__Nat__Interval.lean.pkl 24 theorems
mathlib__Mathlib__LinearAlgebra__Span.lean.pkl 1 theorems
mathlib__Mathlib__Data__List__OfFn.lean.pkl 8 theorems
mathlib__Mathlib__Data__Set__Finite.lean.pkl 71 theorems
mathlib__Mathlib__Data__List__Duplicate.lean.pkl 11 theorems
mathlib__Mathlib__Analysis__Convex__Cone__Extension.lean.pkl 4 theorems
mathlib__Mathlib__Data__ENNReal__Real.lean.pkl 54 theorems
mathlib__Mathlib__Tactic__Sat__FromLRAT.lean.pkl 2 theorems
mathlib__Mathlib__Analysis__Complex__RealDeriv.lean.pkl 13 theorems
mathlib__Mathlib__Algebra__GCDMonoid__Finset.lean.pkl 13 theorems
mathlib__Mathlib__Combinatorics__Composition.lean.pkl 38 theorems
mathlib__Mathlib__RingTheory__AdjoinRoot.lean.pkl 17 theorems
mathlib__Mathlib__Data__Set__Image.lean.pkl 90 theorems
mathlib__Mathlib__Data__Set__Intervals__Basic.lean.pkl 60 theorems
mathlib__Mathlib__Topology__Bornology__Basic.lean.pkl 7 theorems
mathlib__Mathlib__Topology__Instances__AddCircle.lean.pkl 15 theorems

In [4]:
traced_file = traced_repo.get_traced_file('src/algebra/algebra/basic.lean')

print(traced_file)

TracedFile(root_dir=PosixPath('/home/mcwave/.cache/lean_dojo/leanprover-community-mathlib3-65a1391a0106c9204fe45bc73a039f056558cb83/mathlib3'), repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), lean_file=LeanFile(path=PosixPath('src/algebra/algebra/basic.lean'), uses_lean4=False))


In [5]:
traced_file.get_premise_definitions()

[{'full_name': 'algebra',
  'code': "@[nolint has_nonempty_instance]\nclass algebra (R : Type u) (A : Type v) [comm_semiring R] [semiring A]\n  extends has_smul R A, R →+* A :=\n(commutes' : ∀ r x, to_fun r * x = x * to_fun r)\n(smul_def' : ∀ r x, r • x = to_fun r * x)",
  'start': [113, 1],
  'end': [117, 42],
  'kind': 'class'},
 {'full_name': 'algebra_map',
  'code': 'def algebra_map (R : Type u) (A : Type v) [comm_semiring R] [semiring A] [algebra R A] : R →+* A :=\nalgebra.to_ring_hom',
  'start': [121, 1],
  'end': [122, 20],
  'kind': 'definition'},
 {'full_name': 'algebra_map.has_lift_t',
  'code': 'def has_lift_t (R A : Type*) [comm_semiring R] [semiring A] [algebra R A] :\n  has_lift_t R A := ⟨λ r, algebra_map R A r⟩',
  'start': [126, 1],
  'end': [127, 45],
  'kind': 'definition'},
 {'full_name': 'algebra_map.coe_zero',
  'code': '@[simp, norm_cast] lemma coe_zero : (↑(0 : R) : A) = 0',
  'start': [135, 1],
  'end': [135, 85],
  'kind': 'theorem'},
 {'full_name': 'algebra_m

In [6]:
thm = traced_file.get_traced_theorem("algebra.left_comm")

print(thm.theorem)

proof_node = thm.get_proof_node()
proof = proof_node.lean_file[proof_node.start : proof_node.end]
print(proof)

traced_tactics = thm.get_traced_tactics()
print("\ntraced_tactics:\n", traced_tactics)

tac = traced_tactics[0]
print("\ntac:\n", tac)

Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/algebra/algebra/basic.lean'), full_name='algebra.left_comm')
by rw [← mul_assoc, ← commutes, mul_assoc]

traced_tactics:
 [TracedTactic(tactic=rw [← mul_assoc, ← commutes, mul_assoc], state_before=R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y), state_after=no goals)]

tac:
 TracedTactic(tactic=rw [← mul_assoc, ← commutes, mul_assoc], state_before=R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y), state_after=no goals)


In [3]:
from utils.lean_math_utils import *

#theorem = Theorem(repo, "MIL/C02_Basics/solutions/Solutions_S04_More_on_Order_and_Divisibility.lean",
#                 "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex1")

theorem = Theorem(repo, 'src/algebra/algebra/basic.lean', "algebra.left_comm")

print("Entering")
dojo, state_0 = Dojo(theorem).__enter__()
print("Done")

Entering


2024-09-05 21:52:04.482 | WARNING  | lean_dojo.interaction.dojo:__enter__:227 - Docker is strongly recommended when using LeanDojo with Lean 3. See https://leandojo.readthedocs.io/en/latest/user-guide.html#advanced-running-within-docker.


Done


In [4]:
state_1 = dojo.run_tac(state_0, "rw [← mul_assoc]")

#print(state_1)
print(state_1.pp)
classify_lean_elements(state_1.pp)

R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * ⇑(algebra_map R A) r * y = ⇑(algebra_map R A) r * (x * y)


({'R': 'type',
  'A': 'type',
  '_inst_1': 'unknown',
  '_inst_3': 'unknown',
  '_inst_4': 'unknown',
  'x': 'unknown',
  'r': 'unknown',
  'y': 'variable'},
 {'R': 'Type u,',
  'A': 'Type w,',
  '_inst_1': 'comm_semiring R,',
  '_inst_3': 'semiring A,',
  '_inst_4': 'algebra R A,',
  'x': 'A,',
  'r': 'R,',
  'y': 'A'})

In [5]:
state_2 = dojo.run_tac(state_1, "rw [← commutes]") 

print(state_2.pp)

R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ ⇑(algebra_map R A) r * x * y = ⇑(algebra_map R A) r * (x * y)


In [6]:
state_3 = dojo.run_tac(state_2, "rw [mul_assoc]") 

print(state_3)

ProofFinished(tactic_state_id=3, message='')


In [ ]:
# tacs = ['rw [isCoprime_comm] at H H ⊢',
#    'rw [mul_comm] at H',
#    'rw [isCoprime_comm] at H H ⊢',
#    'rw [isCoprime_comm] at H ⊢',
#    'rw [isCoprime_comm] at H H ⊢',
#    'intro y nvar0 nvar1 nvar2 nvar3',
#    'rw [isCoprime_comm] at H H ⊢',
#    'exact H.of_mul_left_left']

# file_path = '.lake/packages/mathlib/Mathlib/RingTheory/Coprime/Basic.lean'
# full_name = 'IsCoprime.of_mul_right_left'

# theorem = Theorem(repo, file_path, full_name)

# dojo, state_0 = Dojo(theorem).__enter__()
# curr_state = state_0

# for tac in tacs:
#     print()
#     print(tac)
#     next_state = dojo.run_tac(curr_state, tac)
#     print(next_state.pp)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

class TripletDataset(Dataset):
    def __init__(self, triplets, tokenizer):
        self.triplets = triplets
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, index):
        q, s, n = self.triplets[index]
        q_encoded = self.tokenizer(q, padding='max_length', max_length=256, truncation=True, return_tensors='pt')
        s_encoded = self.tokenizer(s, padding='max_length', max_length=64, truncation=True, return_tensors='pt')
        n_encoded = self.tokenizer(n, padding='max_length', max_length=64, truncation=True, return_tensors='pt')
        return q_encoded, s_encoded, n_encoded

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, q_embed, s_embed, n_embed):
        dist_pos = torch.norm(q_embed - s_embed, p=2, dim=1)
        dist_neg = torch.norm(q_embed - n_embed, p=2, dim=1)
        loss = torch.mean(torch.relu(dist_pos - dist_neg + self.margin))
        return loss

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model_state = torch.load('/home/mcwave/code/axiomatization/lean3/datasets/rag_tactic_templates/bert_embeder_state-batch64-7k-loss0066.model')
model_tac   = torch.load('/home/mcwave/code/axiomatization/lean3/datasets/rag_tactic_templates/bert_embeder_tac-batch64-7k-loss0066.model')
device = model_state.device
print(device)

/home/mcwave/anaconda3/envs/lean3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


cuda:0


In [8]:
import json

import faiss

index = faiss.read_index('/home/mcwave/code/axiomatization/lean3/datasets/rag_tactic_templates/faiss_index_bert_embeds-batch64-60k-loss0066.idx')

fin = open('/home/mcwave/code/axiomatization/lean3/datasets/rag_tactic_templates/tac_template_freq.json', 'r')
tac_template_freq = json.load(fin)
fin.close()

all_tacs = list(tac_template_freq.keys())

In [10]:
# Example query string

def get_similar_tacs(query_string, model_state, tokenizer, faiss_index, all_tacs, num_returned=100, verbose=False):
    # Convert query string to BERT embedding
    device = model_state.device
    with torch.no_grad():
        encoded_input = tokenizer(query_string, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = model_state(**encoded_input)
        query_embedding = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()

    # Perform nearest neighbor search
    distances, indices = faiss_index.search(query_embedding, num_returned)

    if verbose:
        # Print the nearest neighbors and their distances
        print(f"Nearest neighbors for: {query_string}")
    output = []
    for i in range(num_returned):
        idx = indices[0][i]
        distance = distances[0][i]
        similarity = 1.0 / (1 + distance*distance/1000)
        tac = all_tacs[idx]
        output.append((all_tacs[idx],similarity))
        if verbose:
            print(f"{i+1}. {all_tacs[idx]} (similarity: {similarity})")
    return sorted(output, key=lambda item: item[1], reverse=True)

code = 'theorem left_comm (x : A) (r : R) (y : A) : x * (algebra_map R A r * y) = algebra_map R A r * (x * y)'
state_pp = state_0.pp
query = code + ' # ' + state_pp
print("QUERY:\n", query)

suggestions = get_similar_tacs(query, model_state, tokenizer, index, all_tacs, num_returned=200)
for i in range(len(suggestions)):
    print(suggestions[i])
#     if 'dvd_mul_of_dvd_left' in suggestions[i][0] :
#         print("Found at", i, ":", suggestions[i][0])

QUERY:
 theorem left_comm (x : A) (r : R) (y : A) : x * (algebra_map R A r * y) = algebra_map R A r * (x * y) # R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y)
('rw [(algebra_map _ _).map_mul]', 0.01508938930647988)
('simp_rw [algebra_map_smul]', 0.014326836784422073)
('simp_rw [map_mul {unknown}]', 0.01400598690287034)
('rw [map_algebra_map_mul]', 0.01390344405688594)
('simp [algebra_map_eq]', 0.013826093878961405)
('simp [algebra_map_eq_smul_one]', 0.013762474527326199)
('simp_rw [algebra.algebra_map_eq_smul_one]', 0.01370012712219518)
('simp [map_mul]', 0.013670050894240263)
('simp_rw [alg_hom.map_mul]', 0.013641544129009787)
('rw [mul_comm _ _⟩)]', 0.013410964658021973)
('rw [←map_mul (as_algebra_hom {unknown})]', 0.013252265360876786)
('simp [algebra.algebra_map_eq_smul_one]', 0.013229414901220198)
('convert (mul_assoc _ _ _).symm', 0.01309235892088

In [11]:
import pickle

print("Loading traced theorems ...")
fin = open('/home/mcwave/code/axiomatization/lean3/datasets/train_traced_theorems_repo_mathlib3.pkl', 'rb')
train_traced_theorems = pickle.load(fin)
fin.close()

Loading traced theorems ...


In [12]:
import lean_dojo
import random
from random import randint
import time
import shutil

from utils.lean_math_utils import *


#theorem = Theorem(repo, "MIL/C02_Basics/solutions/Solutions_S05_Proving_Facts_about_Algebraic_Structures.lean",
#                  "Solutions_S05_Proving_Facts_about_Algebraic_Structures_ex1")
#theorem = Theorem(repo, "MIL/C02_Basics/solutions/Solutions_S01_Calculating.lean",
#                  "Solutions_S01_Calculating_ex2")

MAX_MEMORY_USAGE = 16*1024*1024*1024
MAX_STEPS = 40000
MAX_TACTIC_FROM_TEMPLATE = 50
PENALTY_SEEN_TARGET_MULTIPLIER = 3
MAX_NUM_DOJO_ATTEMPT = 2


# The complexity of a state if calculated from lengths of its targets
# seen_target_freq is a dict{string, int} of strings containing lines of targets that have been seen
# If all targets in the current state has been seen, add to complexity PENALTY_SEEN_TARGET_MULTIPLIER*min_freq
# At the same time, seen_target_freq is updated by the targets in this state
# if base_complexity is not none, complexity is base_complexity+1
def explore_state_complexity(state, base_complexity=None, seen_target_freq=None):
    if '⊢ False' in state.pp:
        return 1000000
    if base_complexity is not None:
        return base_complexity + 1
    complexity = 0
    lines = state.pp.split('\n')
    targets = []
    min_freq = 1000000
    for line in lines:
        if line.startswith("⊢"):
            target = line[1:].strip()
            targets.append(target)
            if seen_target_freq is not None:
                if target in seen_target_freq:
                    min_freq = min(min_freq, seen_target_freq[target])
                    seen_target_freq[target] = seen_target_freq[target] + 1
                else:
                    min_freq = 0
                    seen_target_freq[target] = 1
    lengths = [len(x) for x in targets]
    complexity = max(lengths)/2 + sum(lengths)/2
    if seen_target_freq is not None:
        complexity += PENALTY_SEEN_TARGET_MULTIPLIER*min_freq
    return complexity

# Print the trace from state_0 to state
def get_tactic_trace(curr_state, state_dict):
    if type(curr_state) == lean_dojo.interaction.dojo.ProofFinished:
        _, parent_states, tactics = state_dict[curr_state]
    else:
        _, parent_states, tactics = state_dict[curr_state.pp]
    if len(parent_states) == 0:
        return []
    else:
        return get_tactic_trace(parent_states[0], state_dict) + [(tactics[0], curr_state)]
    
# Create the inverse tactic for a 'rw' tactic template
def get_inverse_tactic(tac_template):
    if not tac_template.startswith('rw') or not '[' in tac_template \
        or '←' in tac_template or '[{' in tac_template:
        return None
    result = tac_template.replace('[', '[← ')
    if ',' in result:
        pos = result.find(',')
        result = result[0:pos] + ']'
    return result

@timeout(1)
def dojo_run_tac(dojo, state, tactic):
    #     # Get the current process object
    #     process = psutil.Process(os.getpid())
    #     # Memory info (in bytes)
    #     mem_info = process.memory_info()
    #     # Print the RSS (Resident Set Size): total physical memory used
    #     print(f"Current memory RSS usage: {mem_info.rss / (1024 ** 2):.2f} MB", randint(0,100), end="")
    #print("Run tac", end="")
    result = dojo.run_tac(state, tactic)
    #print("Done")
    return result

#@profile
def explore_states(dojo,
                   state_0,
                   theorem,
                   model_state,
                   tokenizer,
                   rag_index,
                   all_tacs, 
                   theorem_code,
                   traced_tactics=None,
                   max_steps=MAX_STEPS, 
                   max_time=1200, 
                   exit_on_finish=True,
                   verbose=False):
    start_time = time.time()

    #if verbose: print(state_0.pp)
    unused_vars = []
    if theorem_code is not None:
        tokens = tokenize_lean_tactic(theorem_code)
        tokens = [x for x in tokens if x.strip() != '']
        type_of_item, def_of_item = classify_lean_elements(state_0.pp)
        unused_vars = [x for x in type_of_item.keys() if x not in tokens]
        #print(unused_vars)

    state_queue = PriorityQueue() # PQ of states, priority being complexity of the state
    state_dict = {} # Keyed by state.pp (or state if it is ProofFinished). Value is (state, its parent states, and tactics from parent state to this state)

    curr_state = state_0
    base_complexity = 0
    state_queue.push(curr_state, explore_state_complexity(curr_state, base_complexity=base_complexity) + randint(0, 4))
    base_complexity += 1
    state_dict[curr_state.pp] = (curr_state, list(), list())
    
    # Follow traced_tactics and populate state_dict with the authoratative proof
    if traced_tactics is not None:
        for traced_tactic in traced_tactics:
            tactic = traced_tactic.tactic
            try:
                test_state = dojo_run_tac(dojo, curr_state, tactic)
            except:
                break
            if type(test_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
                break
            elif type(test_state) == lean_dojo.interaction.dojo.ProofFinished:
                state_dict[test_state] = (test_state, [curr_state], [tactic])
                if verbose: print("Sucessfully followed proof")
                break
            else:
                state_dict[test_state.pp] = (test_state, [curr_state], [tactic])
                curr_state = test_state
    
    n_steps = 0
    theorem_proven = False
    while state_queue.size() > 0:
        curr_state = state_queue.pop()
        if not hasattr(curr_state, 'pp'):
            continue
        #print("POP STATE: ", base_complexity, curr_state.pp, "\nEND STATE")
        type_of_item, def_of_item = classify_lean_elements(curr_state.pp)
        for var in unused_vars:
            if var in type_of_item:
                del type_of_item[var]
            if var in def_of_item:
                del def_of_item[var]
        if verbose:
            if n_steps == 0:
                print("INIT_STATE:", curr_state.pp)
                print(type_of_item)
        #
        #TODO: Add {'nvar0': 'nvar0', 'nvar1': 'nvar1', ...} to type_of_item
        #
        tactic_set = set()
        query = theorem_code + ' # ' + curr_state.pp
        suggestions = get_similar_tacs(query, model_state, tokenizer, rag_index, all_tacs, num_returned=200)
        tac_templates = [x[0] for x in suggestions]
        #
        # Add inverse tactics of any 'rw' tactics in the set
        inv_tac_templates = []
        for tac_template in tac_templates:
            inv_template = get_inverse_tactic(tac_template)
            if inv_template is not None and inv_template not in tac_templates:
                tac_templates.append(inv_template)
        #
        for tac_template in tac_templates:
            try:
                tactics = generate_tactics_from_template(tac_template, type_of_item)
                if len(tactics) > MAX_TACTIC_FROM_TEMPLATE:
                    tactics = random.sample(tactics, MAX_TACTIC_FROM_TEMPLATE)
            except:
                #print("Cannot generate tactics using", tac_template, ":", type_of_item)
                continue
            for tactic in tactics:
                tactic_set.add(tactic)
        #
        proof_finished = False
        tactic_trace = None
        #print(len(tactic_set), "tactics")
        #print(tactic_set)
        for tactic in tactic_set:
            n_steps += 1
            if n_steps % 1000 == 0:                
                print(n_steps, "steps executed")
                #print(curr_state)
                #print(tactic)
            if n_steps > max_steps:
                break
            num_attempt = 0
            while num_attempt < MAX_NUM_DOJO_ATTEMPT:
                try:
                    test_state = dojo_run_tac(dojo, curr_state, tactic)
                    break
                except Exception as e:
                    num_attempt += 1
            if num_attempt == MAX_NUM_DOJO_ATTEMPT:
                continue
            #print("TRY TAC:", tactic)
            if type(test_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
                continue
            elif type(test_state) == lean_dojo.interaction.dojo.ProofFinished:
                proof_finished = True
                if verbose and not theorem_proven:
                    print("ProofFinished")
#                     tactic_trace = get_tactic_trace(curr_state, state_dict) + [(tactic, test_state)]
#                     for i in range(len(tactic_trace)):
#                         tactic, state = tactic_trace[i]
#                         print("STEP", i)
#                         if tactic != None:
#                             print("Tactic:", tactic)
#                         if hasattr(state, 'pp'):
#                             print(state.pp)
                if test_state in state_dict:
                    _, parent_states, tactics = state_dict[test_state]
                    state_dict[test_state] = (test_state, parent_states + [curr_state], tactics + [tactic])
                else:
                    complexity = 100000000
                    state_queue.push(test_state, complexity)
                    state_dict[test_state] = (test_state, [curr_state], [tactic])
                if exit_on_finish: break
            else:
                #print("TAC:", tactic, "STATE:", test_state.pp)
                if test_state.pp in state_dict:
                    _, parent_states, tactics = state_dict[test_state.pp]
                    state_dict[test_state.pp] = (test_state, parent_states + [curr_state], tactics + [tactic])
                else:
                    complexity = explore_state_complexity(test_state, base_complexity=base_complexity)
                    base_complexity += 1
                    state_queue.push(test_state, complexity  + randint(0, 4))
                    state_dict[test_state.pp] = (test_state, [curr_state], [tactic])
        #
        if proof_finished:
            theorem_proven = True
            if exit_on_finish: break
        if n_steps > max_steps:
            break
        cur_time = time.time()
        if (cur_time - start_time) > max_time:
            print("Max run-time exceeded")
            break
    #
    dojo.__exit__(None, None, None)
    try:
        shutil.rmtree(dojo.tmp_dir)
        print(dojo.tmp_dir, "removed successfully.")
    except Exception as e:
        print(f"An error occurred when removing tmp dir: {e}")
    return state_dict, theorem_proven, tactic_trace

file_path = 'src/algebra/algebra/basic.lean'
full_name = "algebra.left_comm"
theorem = Theorem(repo, file_path, full_name)
# traced_file = traced_repo.get_traced_file(file_path)
# thm = traced_file.get_traced_theorem(full_name)
# for premise in traced_file.get_premise_definitions():
#     if premise['full_name'] == full_name:
#         theorem_code = premise['code']

traced_theorems = train_traced_theorems[file_path]
thm = traced_theorems[full_name]
theorem_code = thm.comments[0]
traced_tactics = thm.get_traced_tactics()
print("traced_tactics:", traced_tactics)

theorem = Theorem(repo, file_path, full_name)

print(theorem_code)

random.seed(42)
dojo, state_0 = Dojo(theorem).__enter__()

state_dict, theorem_proven, tactic_trace = \
    explore_states(dojo,
                   state_0,
                   theorem,
                   model_state,
                   tokenizer,
                   index,
                   all_tacs,
                   theorem_code=theorem_code, 
                   traced_tactics=traced_tactics,
                   max_steps = 200000,
                   exit_on_finish=False,
                   verbose=True)

print(theorem_proven)
print(tactic_trace)

traced_tactics: [TracedTactic(tactic=rw [← mul_assoc, ← commutes, mul_assoc], state_before=R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y), state_after=no goals)]
theorem left_comm (x : A) (r : R) (y : A) :
  x * (algebra_map R A r * y) = algebra_map R A r * (x * y)


2024-09-05 21:56:32.120 | WARNING  | lean_dojo.interaction.dojo:__enter__:227 - Docker is strongly recommended when using LeanDojo with Lean 3. See https://leandojo.readthedocs.io/en/latest/user-guide.html#advanced-running-within-docker.


Sucessfully followed proof
INIT_STATE: R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y)
{'R': 'type', 'A': 'type', 'x': 'unknown', 'r': 'unknown', 'y': 'variable'}
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
1000 steps executed
2000 steps executed
3000 steps executed
4000 steps executed
5000 steps executed
6000 steps executed
7000 steps executed
8000 steps executed
9000 steps executed
10000 steps executed
11000 steps executed
12000 steps executed
13000 steps executed
14000 steps executed
15000 steps executed
16000 steps executed
17000 steps executed
18000 steps executed
19000 steps executed
20000 steps executed
21000 steps executed
22000 steps executed
23000 steps executed
24000 steps executed
25000 steps 

In [18]:
train_traced_theorems[file_path]

{'algebra.left_comm': TracedTheorem(theorem=Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/algebra/algebra/basic.lean'), full_name='algebra.left_comm')),
 'algebra.right_comm': TracedTheorem(theorem=Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/algebra/algebra/basic.lean'), full_name='algebra.right_comm')),
 "algebra.bit0_smul_one'": TracedTheorem(theorem=Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/algebra/algebra/basic.lean'), full_name="algebra.bit0_smul_one'")),
 "algebra.bit1_smul_one'": TracedTheorem(theorem=Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=P

In [13]:
all_states = list(state_dict.values())
print(len(all_states), "states in total")
proven_states = [x[0] for x in all_states if type(x[0]) == lean_dojo.interaction.dojo.ProofFinished]
print(len(proven_states), "proven states")

get_tactic_trace(proven_states[0], state_dict)

1161 states in total
59 proven states


RecursionError: maximum recursion depth exceeded in comparison

In [24]:
parent_1 = state_dict[proven_states[0]]
print(parent_1)
print()
parent_2 = state_dict[parent_1[1][0].pp]
print(parent_2)

print()
parent_3 = state_dict[parent_2[1][0].pp]
print(parent_3)

(ProofFinished(tactic_state_id=194, message=''), [TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime (y * z) x\n⊢ IsCoprime y x', id=17, message='')], ['exact H.of_mul_left_left'])

(TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime (y * z) x\n⊢ IsCoprime y x', id=1263, message=''), [TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime x (y * z)\n⊢ IsCoprime x y', id=0, message=None), TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime x (y * z)\n⊢ IsCoprime x y', id=0, message=None), TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime (y * z) x\n⊢ IsCoprime y x', id=17, message=''), TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime (y * z) x\n⊢ IsCoprime y x', id=17, message=''), TacticState(pp='R : Type u\ninst✝ : CommSemiring R\nx y z : R\nH : IsCoprime (y * z) x\n⊢ IsCoprime y x', id=17, message=''), TacticState(pp='R : Type u\nin

In [14]:
keys = list(state_dict.keys())

for k in range(len(keys)):
    val = state_dict[keys[k]]
    (state, parent_states, tacs) = val
    if 'case' in str(keys[k]):
        continue
    for i in range(len(tacs)):
        if 'mul_left_comm' in tacs[i]:
            print(k)
            print(keys[k])
            print(parent_states[i])
            print(tacs[i])
            print()

23
R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (1 * r • 1 * y) = 1 * r • 1 * (x * y)

R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ mul_one_class A
TacticState(pp='R : Type u,\nA : Type w,\n_inst_1 : comm_semiring R,\n_inst_3 : semiring A,\n_inst_4 : algebra R A,\nx : A,\nr : R,\ny : A\n⊢ x * (1 * r • 1 * y) = 1 * r • 1 * (x * y)\n\nR : Type u,\nA : Type w,\n_inst_1 : comm_semiring R,\n_inst_3 : semiring A,\n_inst_4 : algebra R A,\nx : A,\nr : R,\ny : A\n⊢ mul_one_class A', id=49, message='')
simp only [← mul_smul, smul_assoc, mul_assoc, mul_comm, mul_left_comm] at ⊢ x x

23
R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (1 * r • 1 * y) = 1 * r • 1 * (x * y)

R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : 

In [ ]:
# Check which theorems can be proven

import pickle

file_paths = [
    "MIL/C02_Basics/solutions/Solutions_S01_Calculating.lean",
    "MIL/C02_Basics/solutions/Solutions_S02_Proving_Identities_in_Algebraic_Structures.lean",
    "MIL/C02_Basics/solutions/Solutions_S03_Using_Theorems_and_Lemmas.lean",
    "MIL/C02_Basics/solutions/Solutions_S04_More_on_Order_and_Divisibility.lean",
    "MIL/C02_Basics/solutions/Solutions_S05_Proving_Facts_about_Algebraic_Structures.lean"
]

# theorems_to_compute = [
#     "MyRing.self_sub",
#     "MyRing.one_add_one_eq_two",
#     "MyRing.two_mul",
#     "MyGroup.mul_right_inv",
#     "MyGroup.mul_one",
#     "MyGroup.mul_inv_rev",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex6",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex7",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex8",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex9",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex10"
# ]

#fout = open('/home/mcwave/code/automath/atp/datasets/provability_raw/C02_Basics_S02part2.pkl', 'wb')

for file_path in file_paths:
    traced_file = traced_repo.get_traced_file(file_path)
    premises = traced_file.get_premise_definitions()
    #
    results = []
    for premise in premises:
#         if file_path != "MIL/C02_Basics/solutions/Solutions_S05_Proving_Facts_about_Algebraic_Structures.lean" and \
#             premise['full_name'] not in theorems_to_compute:
#             continue
        if premise['code'].startswith('theorem '):
            print('THEOREM:', premise['full_name'])
            print(premise['code'])
            #theorem = Theorem(repo, file_path, premise['full_name'])
            thm = traced_file.get_traced_theorem(premise['full_name'])
            theorem = thm.theorem
            print("theorem loaded")
            state_dict, theorem_proven, tactic_trace = \
            explore_states(theorem,
                           index,
                           tacs,
                           theorem_code=premise['code'],
                           max_steps = 20000,
                           exit_on_finish=True,
                           verbose=True)
            print("Proved?:", theorem_proven)
            print(tactic_trace)
                
#fout.close()

2024-06-01 20:15:35.476 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


THEOREM: Solutions_S01_Calculating_ex1
theorem Solutions_S01_Calculating_ex1 (a b c : ℝ) : c * b * a = b * (a * c)
theorem loaded
INIT_STATE: a b c : ℝ
⊢ c * b * a = b * (a * c)
{'a': 'variable', 'b': 'variable', 'c': 'variable'}


2024-06-01 20:15:44.721 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


ProofFinished
STEP 0
Tactic: ring
Proved?: True
[('ring', ProofFinished(tactic_state_id=27, message=''))]
THEOREM: Solutions_S01_Calculating_ex2
theorem Solutions_S01_Calculating_ex2 (a b c : ℝ) : a * (b * c) = b * (a * c)
theorem loaded


2024-06-01 20:15:53.717 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


...

20000 steps executed
Proved?: False
None
THEOREM: C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex4
theorem Solutions_S04_More_on_Order_and_Divisibility_ex4 : |a| - |b| ≤ |a - b|
theorem loaded
INIT_STATE: a b c d : ℝ
⊢ |a| - |b| ≤ |a - b|
{'a': 'variable', 'b': 'variable'}
10000 steps executed


2024-06-01 20:48:59.738 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


20000 steps executed
Proved?: False
None
THEOREM: C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex5
theorem Solutions_S04_More_on_Order_and_Divisibility_ex5 : |a| - |b| ≤ |a - b|
theorem loaded
INIT_STATE: a b c d : ℝ
⊢ |a| - |b| ≤ |a - b|
{'a': 'variable', 'b': 'variable'}
10000 steps executed


2024-06-01 20:50:37.769 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


20000 steps executed
Proved?: False
None
THEOREM: C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex6
theorem Solutions_S04_More_on_Order_and_Divisibility_ex6 (h₀ : x ∣ y) (h₁ : y ∣ z) : x ∣ z
theorem loaded
INIT_STATE: w x y z : ℕ
h₀ : x ∣ y
h₁ : y ∣ z
⊢ x ∣ z
{'x': 'unknown', 'y': 'unknown', 'z': 'unknown', 'h₀': 'unknown', 'h₁': 'unknown'}
10000 steps executed


In [16]:
# Generate state pairs and their distances. 
# Probably useful if we hope to generate intermediate states (e.g., "have").
def yield_state_pairs(curr_state, leaf_state, state_dict, seen_states_pp, tactic_list, max_distance=8, max_parents=10):
    #     print("curr_state:", curr_state.pp)
    #     print("leaf_state:", leaf_state.pp)
    if type(curr_state) == lean_dojo.interaction.dojo.ProofFinished:
        _, parent_states, tactics = state_dict[curr_state]
    else:
        _, parent_states, tactics = state_dict[curr_state.pp]
    if len(seen_states_pp) > 1:
        yield (curr_state, leaf_state, len(seen_states_pp) - 1, tactic_list)
    if len(seen_states_pp) <= max_distance:
        num_parents_checked = 0
        for i in range(len(parent_states)):
            parent_state = parent_states[i]
            if parent_state.pp not in seen_states_pp:
                yield from yield_state_pairs(parent_state, 
                                             leaf_state, 
                                             state_dict, 
                                             seen_states_pp + [parent_state.pp],
                                             [tactics[i]] + tactic_list
                                            )
                num_parents_checked += 1
                if num_parents_checked >= max_parents:
                    break
    return

@timeout(3000)
def get_state_provability_data(dojo,
                               state_0,
                               theorem,
                               model_state,
                               tokenizer,
                               rag_index, 
                               all_tacs,
                               theorem_code=None, 
                               traced_tactics=None,
                               max_steps=100000, 
                               negative_ratio=1.0,
                               verbose=False):
    MAX_PROVEN_STATES = 2000
    state_dict, theorem_proven, tactic_trace = \
    explore_states(dojo,
                   state_0,
                   theorem,
                   model_state,
                   tokenizer,
                   rag_index, 
                   all_tacs,
                   theorem_code=theorem_code, 
                   traced_tactics=traced_tactics,
                   max_steps=max_steps,
                   exit_on_finish=False,
                   verbose=verbose)
    #
    all_states = list(state_dict.values())
    print(len(all_states), "states in total")
    proven_states = [x[0] for x in all_states if type(x[0]) == lean_dojo.interaction.dojo.ProofFinished]
    print(len(proven_states), "proven states")
    state_pairs = []
    seen_state_pps = set()
    #
    for i in range(len(proven_states)):
        proven_state = proven_states[i]
        #print(proven_state)
        pairs = list(yield_state_pairs(proven_state, proven_state, state_dict, [str(proven_state)], []))
        ancestor_state_dict = {}
        for ancestor, leaf, distance, tactic in pairs:
            if ancestor.pp not in ancestor_state_dict:
                ancestor_state_dict[ancestor] = (distance, tactic)
            else:
                if distance < ancestor_state_dist[ancestor]:
                    ancestor_state_dict[ancestor] = (distance, tactic)
        #print(ancestor_state_dist)
        #print(len(ancestor_state_dict), "ancestors found")
        for ancestor, (distance, tactic) in ancestor_state_dict.items():
            state_pairs.append((ancestor, proven_state, distance, tactic))
            seen_state_pps.add(ancestor.pp)
        if i % 100 == 0 and i > 0:
            print(i, "proven states processed")
        if i > MAX_PROVEN_STATES:
            break
    #
    num_positive = len(state_pairs)
    # Add negative examples
    if num_positive > 0:
        num_negative = 0
        for tup in all_states:
            state = tup[0]
            if hasattr(state, 'pp') and state.pp not in seen_state_pps:
                state_pairs.append((state, random.choice(proven_states), -1, None))
                num_negative += 1
                if num_negative >= num_positive * negative_ratio:
                    break
    return state_pairs, state_dict

file_path = 'src/algebra/algebra/basic.lean'
full_name = "algebra.left_comm"
theorem = Theorem(repo, file_path, full_name)
# traced_file = traced_repo.get_traced_file(file_path)
# thm = traced_file.get_traced_theorem(full_name)
# for premise in traced_file.get_premise_definitions():
#     if premise['full_name'] == full_name:
#         theorem_code = premise['code']

traced_theorems = train_traced_theorems[file_path]
thm = traced_theorems[full_name]
theorem_code = thm.comments[0]
traced_tactics = thm.get_traced_tactics()
print("traced_tactics:", traced_tactics)

theorem = Theorem(repo, file_path, full_name)

random.seed(42)
dojo, state_0 = Dojo(theorem).__enter__()

state_pairs, state_dict = \
    get_state_provability_data(dojo,
                               state_0,
                               theorem,
                               model_state,
                               tokenizer,
                               index,
                               all_tacs,
                               theorem_code=theorem_code,
                               traced_tactics=traced_tactics,
                               max_steps = 20000,
                               verbose=True)

print(len(state_pairs))

traced_tactics: [TracedTactic(tactic=rw [← mul_assoc, ← commutes, mul_assoc], state_before=R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y), state_after=no goals)]


2024-09-06 09:54:08.104 | WARNING  | lean_dojo.interaction.dojo:__enter__:227 - Docker is strongly recommended when using LeanDojo with Lean 3. See https://leandojo.readthedocs.io/en/latest/user-guide.html#advanced-running-within-docker.


Sucessfully followed proof
INIT_STATE: R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y)
{'R': 'type', 'A': 'type', 'x': 'unknown', 'r': 'unknown', 'y': 'variable'}
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
ProofFinished
1000 steps executed
2000 steps executed
3000 steps executed
4000 steps executed
5000 steps executed
6000 steps executed
7000 steps executed
8000 steps executed
9000 steps executed
10000 steps executed
11000 steps executed
12000 steps executed
13000 steps executed
14000 steps executed
15000 steps executed
16000 steps executed
17000 steps executed
18000 steps executed
19000 steps executed
20000 steps executed
An error occurred when removing tmp dir: [Errno 2] No such file or directory: '/tmp/tmpnvu7e4lb'

In [17]:
state_pairs

[(TacticState(pp='R : Type u,\nA : Type w,\n_inst_1 : comm_semiring R,\n_inst_3 : semiring A,\n_inst_4 : algebra R A,\nx : A,\nr : R,\ny : A\n⊢ x * (⇑(algebra_map R A) r * y) = ⇑(algebra_map R A) r * (x * y)', id=0, message=None),
  ProofFinished(tactic_state_id=1, message=''),
  1,
  ['rw [← mul_assoc, ← commutes, mul_assoc]']),
 (TacticState(pp='R : Type u,\nA : Type w,\n_inst_1 : comm_semiring R,\n_inst_3 : semiring A,\n_inst_4 : algebra R A,\nx : A,\nr : R,\ny : A\n⊢ x * (1 * ⇑(algebra_map R A) r * y) = 1 * ⇑(algebra_map R A) r * (x * y)\n\nR : Type u,\nA : Type w,\n_inst_1 : comm_semiring R,\n_inst_3 : semiring A,\n_inst_4 : algebra R A,\nx : A,\nr : R,\ny : A\n⊢ mul_one_class A', id=2, message=''),
  ProofFinished(tactic_state_id=1, message=''),
  5,
  ['rw [←mul_one y]',
   'have : r * r * (x * r) = x * r * 1 := by rw [mul_one]',
   'have : x * r * (x * x) = r * r * 1 := by rw [mul_one]',
   'rw mul_assoc',
   'rw [← mul_assoc, ← commutes, mul_assoc]']),
 (TacticState(pp='R : Ty

In [43]:
import pickle
import os

def get_filename(filepath):
    # Extract the base name (e.g., 'example.txt' from '/path/to/example.txt')
    base_name = os.path.basename(filepath)
    # Split the base name and the extension and return just the base name
    file_name_without_extension = os.path.splitext(base_name)[0]
    return file_name_without_extension

file_paths = [
    "MIL/C02_Basics/solutions/Solutions_S01_Calculating.lean",
    "MIL/C02_Basics/solutions/Solutions_S02_Proving_Identities_in_Algebraic_Structures.lean",
    "MIL/C02_Basics/solutions/Solutions_S03_Using_Theorems_and_Lemmas.lean",
    "MIL/C02_Basics/solutions/Solutions_S04_More_on_Order_and_Divisibility.lean",
    "MIL/C02_Basics/solutions/Solutions_S05_Proving_Facts_about_Algebraic_Structures.lean"
]

# theorems_to_compute = [
#     "MyRing.self_sub",
#     "MyRing.one_add_one_eq_two",
#     "MyRing.two_mul",
#     "MyGroup.mul_right_inv",
#     "MyGroup.mul_one",
#     "MyGroup.mul_inv_rev",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex6",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex7",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex8",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex9",
#     "C02S04.Solutions_S04_More_on_Order_and_Divisibility_ex10"
# ]

def compute_provability_training_data(file_path, output_path):
    file_name = get_filename(file_path)
    fout = open(output_path + file_name + '.pkl', 'wb')
    traced_file = traced_repo.get_traced_file(file_path)
    premises = traced_file.get_premise_definitions()
    #
    results = []
    for premise in premises:
        if premise['code'].startswith('theorem '):
            print('THEOREM:', premise['full_name'])
            print(premise['code'])
            thm = traced_file.get_traced_theorem(premise['full_name'])
            theorem = thm.theorem
            traced_statistic = thm.get_traced_tactics()
            print("theorem loaded")
            state_pairs, _ = \
                get_state_provability_data(theorem,
                                           index,
                                           tacs,
                                           theorem_code=premise['code'],
                                           traced_tactics=traced_tactics,
                                           max_steps=10000,
                                           verbose=True)
            print(len(state_pairs), "state pairs found")
            #
            for state_pair in state_pairs:
                result = (file_path, premise, theorem, state_pair)
                results.append(result)
                pickle.dump(result, fout)
    #
    fout.close()
    return len(results)
    
    
OUTPUT_FOLDER = '/home/mcwave/code/automath/atp/datasets/provability/rag/'

for file_path in file_paths:
    compute_provability_training_data(file_path, OUTPUT_FOLDER)

2024-06-03 22:17:53.707 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


THEOREM: Solutions_S01_Calculating_ex1
theorem Solutions_S01_Calculating_ex1 (a b c : ℝ) : c * b * a = b * (a * c)
theorem loaded
INIT_STATE: a b c : ℝ
⊢ c * b * a = b * (a * c)
{'a': 'variable', 'b': 'variable', 'c': 'variable'}
ProofFinished
ProofFinished
10000 steps executed
1045 states in total
44 proven states
ProofFinished(tactic_state_id=28, message='')
15 ancestors found
ProofFinished(tactic_state_id=35, message='')
15 ancestors found
ProofFinished(tactic_state_id=93, message='')
15 ancestors found
ProofFinished(tactic_state_id=100, message='')
15 ancestors found
ProofFinished(tactic_state_id=131, message='')
15 ancestors found
ProofFinished(tactic_state_id=138, message='')
15 ancestors found
ProofFinished(tactic_state_id=236, message='')
15 ancestors found
ProofFinished(tactic_state_id=242, message='')
15 ancestors found
ProofFinished(tactic_state_id=305, message='')
15 ancestors found
ProofFinished(tactic_state_id=311, message='')
15 ancestors found
ProofFinished(tactic_state

2024-06-03 22:18:50.442 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


15 ancestors found
ProofFinished(tactic_state_id=1566, message='')
15 ancestors found
ProofFinished(tactic_state_id=1602, message='')
15 ancestors found
ProofFinished(tactic_state_id=1609, message='')
15 ancestors found
ProofFinished(tactic_state_id=1720, message='')
15 ancestors found
ProofFinished(tactic_state_id=1726, message='')
15 ancestors found
1408 state pairs found
THEOREM: Solutions_S01_Calculating_ex2
theorem Solutions_S01_Calculating_ex2 (a b c : ℝ) : a * (b * c) = b * (a * c)
theorem loaded


SystemExit: -1

/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [138]:
import ray

# Initialize Ray cautiously
if not ray.is_initialized():
    ray.init(num_cpus=5)

@ray.remote
def compute_provability_training_data_remote(file_path, output_path):
    compute_provability_training_data(file_path, output_path)

# Submit tasks
result_ids = [compute_provability_training_data_remote.remote(i) for file_path in file_paths]

# Fetch results
results = ray.get(result_ids)

# Optionally, shut down Ray if you're done with all computations
ray.shutdown()

# Display results
print(results)

2024-06-02 22:23:43,060	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(raylet) [2024-06-02 22:24:43,027 E 313863 313863] (raylet) node_manager.cc:3002: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 2c0ea0932736f0540f910b5bb7c16183436a30e051a7aa0d64e2e589, IP: 192.168.0.192) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.192`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero

ValueError: The remote function __main__.compute_provability_training_data_remote is too large (2926 MiB > FUNCTION_SIZE_ERROR_THRESHOLD=95 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.

(raylet) [2024-06-02 22:25:43,027 E 313863 313863] (raylet) node_manager.cc:3002: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 2c0ea0932736f0540f910b5bb7c16183436a30e051a7aa0d64e2e589, IP: 192.168.0.192) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.192`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [19]:
# Create the inverse tactic for a 'rw' tactic template
def get_inverse_tactic(tac_template):
    if not tac_template.startswith('rw') or not '[' in tac_template \
        or '←' in tac_template or '[{' in tac_template:
        return None
    result = tac_template.replace('[', '[← ')
    if ',' in result:
        pos = result.find(',')
        result = result[0:pos] + ']'
    return result

for tac, _ in suggestions:
    print(tac)
    print(get_inverse_tactic(tac))
    print()

rw [mul_comm]
rw [← mul_comm]

rw [mul_assoc]
rw [← mul_assoc]

ring
None

rw [mul_one]
rw [← mul_one]

linarith
None

subst {unknown}
None

> rfl
None

abel
None

norm_cast
None

field_simp
None

norm_num
None

> simp
None

cases {unknown} with
None

symm
None

gcongr
None

positivity
None

simp
None

calc
None

cases {unknown}
None

contradiction
None

rw [mul_add]
rw [← mul_add]

rfl
None

simp_all
None

constructor
None

congr
None

intro
None

simpa
None

rw [← {unknown}]
None

classical
None

simp [*]
None

simp only [{unknown}]
None

ext
None

exfalso
None

rwa [{unknown}]
None

split
None

simp [{unknown}]
None

rw [mul_zero]
rw [← mul_zero]

rw [{unknown}]
None

rw[{unknown}]
None

apply {unknown}
None

>
None

tauto
None

rw [one_mul]
rw [← one_mul]

congr 1
None

dsimp
None

exact {unknown}
None

intros
None

dsimp only
None

push_cast
None

decide
None

by_contra! {nvar0}
None

aesop
None

swap
None

left
None

rw [{hypothesis}]
None

rintro rfl
None

exact
None

rw [{funct

In [5]:
import atexit

import posix_ipc
import time
import threading

class SystemSemaphore:
    def __init__(self, name, limit, max_hold_time=240, acquire_timeout=60):
        self.name = name
        self.limit = limit
        self.max_hold_time = max_hold_time
        self.acquire_timeout = acquire_timeout

    def __enter__(self):
        self.lock = posix_ipc.Semaphore(f'/{self.name}', posix_ipc.O_CREAT, 0x384, self.limit)
        start_time = time.time()
        while True:
            if self.lock.acquire(self.acquire_timeout):
                break
            elapsed_time = time.time() - start_time
            if elapsed_time > self.acquire_timeout:
                raise TimeoutError(f"Failed to acquire semaphore within {self.acquire_timeout} seconds.")

        self.start_time = time.time()
        self.timer = threading.Timer(self.max_hold_time, self.release_semaphore)
        self.timer.start()

    def __exit__(self, _type, value, tb):
        self.timer.cancel()
        self.release_semaphore()

    def release_semaphore(self):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.max_hold_time:
            print(f"Warning: Semaphore was held for {elapsed_time:.2f} seconds, exceeding the maximum hold time of {self.max_hold_time} seconds.")
        self.lock.release()

    def unlink(self):
        posix_ipc.unlink_semaphore(f'/{self.name}')

semaphore = SystemSemaphore('dojo-enter-1', 1)

def cleanup():
    semaphore.unlink()

atexit.register(cleanup)

semaphore.unlink()

ExistentialError: No semaphore exists with the specified name

In [9]:
log_file_path = '/home/mcwave/code/automath/atp/datasets/mathlib4_rag_explore_state_log.txt'

import re

def parse_rag_explore_state_log(log_file_path):
    output = []
    # Regular expression pattern to match PosixPath and full_name
    pattern = r"PosixPath\('(.+?)'\).+full_name='(.+?)'"
    # Open the text file for reading
    with open(log_file_path, 'r') as file:
        # Read the file line by line
        for line in file:
            if 'Start entering ' not in line:
                continue
            # Use re.search to find the matches
            match = re.search(pattern, line)
            if match:
                posix_path = match.group(1)
                full_name = match.group(2)
            else:
                continue
            output.append((posix_path, full_name))
    return output


output = parse_rag_explore_state_log(log_file_path)
print(len(output), "theorems processed")
output

261 theorems processed


[('.lake/packages/mathlib/Mathlib/RingTheory/Coprime/Basic.lean',
  'isCoprime_self'),
 ('.lake/packages/mathlib/Mathlib/Algebra/Group/Conj.lean',
  'ConjClasses.map_surjective'),
 ('.lake/packages/mathlib/Mathlib/Data/Nat/Squarefree.lean',
  'Nat.squarefree_iff_nodup_factors'),
 ('.lake/packages/mathlib/Mathlib/GroupTheory/GroupAction/Prod.lean',
  'Prod.smul_zero_mk'),
 ('.lake/packages/mathlib/Mathlib/Algebra/GroupWithZero/Divisibility.lean',
  'mul_dvd_mul_iff_left'),
 ('.lake/packages/mathlib/Mathlib/GroupTheory/Perm/Sign.lean',
  'Equiv.Perm.perm_inv_on_of_perm_on_finset'),
 ('.lake/packages/mathlib/Mathlib/RingTheory/Coprime/Basic.lean',
  'isCoprime_zero_left'),
 ('.lake/packages/mathlib/Mathlib/Data/Nat/Squarefree.lean',
  'Squarefree.natFactorization_le_one'),
 ('.lake/packages/mathlib/Mathlib/Algebra/Group/Conj.lean',
  'isConj_iff_conjugatesOf_eq'),
 ('.lake/packages/mathlib/Mathlib/Algebra/Group/Conj.lean',
  'ConjClasses.mem_carrier_iff_mk_eq'),
 ('.lake/packages/mathlib/

In [4]:
import pickle

results = []
#fin = open('/home/mcwave/code/automath/atp/datasets/provability/rag/mathlib__Mathlib__RingTheory__Coprime__Basic.lean.pkl', 'rb')
fin = open('/home/mcwave/code/automath/atp/datasets/provability/rag/mathlib__Mathlib__Data__Nat__PrimeFin.lean.pkl', 'rb')
while True:
    try:
        result = pickle.load(fin)
    except Exception as e:
        print(e)
        break
    file_path, full_name, theorem, state_pair = result
    results.append(result)

fin.close()

Ran out of input


In [5]:
len(results)

7919

In [6]:
theorems = set()
for file_path, full_name, theorem, state_pair in results:
    theorems.add(full_name)
print(theorems)

{'Nat.mem_primeFactors_of_ne_zero'}


In [10]:
results[2]

('.lake/packages/mathlib/Mathlib/Data/Nat/PrimeFin.lean',
 'Nat.mem_primeFactors_of_ne_zero',
 Theorem(repo=LeanGitRepo(url='https://github.com/xiaoxin-yin/math-in-lean', commit='20077bcd4392317ddb9605404fda3a85e40e8956'), file_path=PosixPath('.lake/packages/mathlib/Mathlib/Data/Nat/PrimeFin.lean'), full_name='Nat.mem_primeFactors_of_ne_zero'),
 (TacticState(pp='a b k m n p : ℕ\nhn : n ≠ 0\n⊢ p ∈ n.primeFactors ↔ Irreducible p ∧ p ∣ n', id=21, message=''),
  ProofFinished(tactic_state_id=1, message=''),
  3,
  ['rw [Nat.irreducible_iff_prime]', 'simp_rw [← prime_iff]', 'simp [hn]']))

In [8]:
def is_pp_useful(pp):
    lines = pp.split('\n')
    targets = []
    conditions = []
    min_freq = 1000000
    for line in lines:
        if line.startswith("⊢"):
            target = line.strip()
            targets.append(target)
        else:
            conditions.append(line.strip())
    if '⊢ False' in targets:
        return False
    if '⊢ R' in targets:
        return False
    if '?m.' in pp:
        return False
    return True

num_output = 0
for i in range(len(results)):
    file_path, full_name, theorem, state_pair = results[i]
    if full_name != 'Nat.mem_primeFactors_of_ne_zero': #'IsCoprime.of_mul_right_left': #'IsCoprime.mul_right':
        continue
    if not is_pp_useful(results[i][3][0].pp):
        continue
    print("CASE", i)
    print(results[i][3][0].pp)
    print(theorem)
    print()
    num_output += 1
    if num_output > 400:
        break

CASE 0
a b k m n p : ℕ
hn : n ≠ 0
⊢ p ∈ n.primeFactors ↔ Prime p ∧ p ∣ n
Theorem(repo=LeanGitRepo(url='https://github.com/xiaoxin-yin/math-in-lean', commit='20077bcd4392317ddb9605404fda3a85e40e8956'), file_path=PosixPath('.lake/packages/mathlib/Mathlib/Data/Nat/PrimeFin.lean'), full_name='Nat.mem_primeFactors_of_ne_zero')

CASE 1
a b k m n p : ℕ
hn : n ≠ 0
⊢ p ∈ n.primeFactors ↔ _root_.Prime p ∧ p ∣ n
Theorem(repo=LeanGitRepo(url='https://github.com/xiaoxin-yin/math-in-lean', commit='20077bcd4392317ddb9605404fda3a85e40e8956'), file_path=PosixPath('.lake/packages/mathlib/Mathlib/Data/Nat/PrimeFin.lean'), full_name='Nat.mem_primeFactors_of_ne_zero')

CASE 2
a b k m n p : ℕ
hn : n ≠ 0
⊢ p ∈ n.primeFactors ↔ Irreducible p ∧ p ∣ n
Theorem(repo=LeanGitRepo(url='https://github.com/xiaoxin-yin/math-in-lean', commit='20077bcd4392317ddb9605404fda3a85e40e8956'), file_path=PosixPath('.lake/packages/mathlib/Mathlib/Data/Nat/PrimeFin.lean'), full_name='Nat.mem_primeFactors_of_ne_zero')

CASE 3
a b k